In [15]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)

import aiohttp
import asyncio
%autoawait asyncio

import re
from datetime import datetime

import bs4
from bs4 import BeautifulSoup
from db.config import async_session
from db.dals.company_dal import Company
from utilities.time_measure import timeit
from apps.ipo.agents import get_user_agents
from config.config_log import logging

logger = logging.getLogger("info-logger")


async def delist(session, company_name):

    instance = Company(session)
    read, ci_demand_forcast_date = instance.read(company_name)

    # DB에는 공모철회가 없지만 공모철회로 데이터를 바꾸어야함.
    if read and ci_demand_forcast_date != "공모철회":
        delisted = instance.delist(company_name)
        if delisted:
            logger.info("공모철회 한 회사 : {company_name}")


async def get_name_n_code(td_name):
    try:
        a_tag = td_name.select_one('a[href*="view_pg"][class=""]')
        href = a_tag["href"]
        company_code = re.search("code=(.+)&", href).group(1)
        company_name = a_tag.select_one("font").text.strip().replace(".", "")
        return company_code, company_name
    except KeyError as err:
        logger.error(err)
    except AttributeError as err:
        logger.error(err)


async def get_years(year):
    current_year = datetime.now().year
    for k in range(year, current_year + 1):
        yield k


from async_retrying import retry


@retry(attempts=100)
async def scrape_company_codes(year=2021):
    ipo_companies = []
    delisted_companies = []
    header = await get_user_agents()
    async for year in get_years(year):
        logger.debug("연도", year)
        page_data = None
        page = 1
        while page_data is None:
            ipo_companies_in_page = 0
            delisted_companies_in_page = 0
            url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
            logger.debug(f"page: {page}, url: {url}")
            page += 1

            try:
                async with aiohttp.ClientSession() as session:
                    async with session.get(url, headers=header) as resp:
                        soup = BeautifulSoup(await resp.text(), "lxml")

            except (aiohttp.ClientError, asyncio.TimeoutError) as e:
                logger.error("Request failed, retrying in 5 seconds...")
                logger.error(e)
                await asyncio.sleep(1)

            page_data = soup.select_one('td[colspan="9"]')
            if page_data is not None:
                logger.debug("페이지에 기업이 없습니다. 다음 연도로 전환됩니다")
                break
            table = soup.select_one(
                "#print > table >  tr:nth-child(4) > td > table >  tr:nth-child(3) > td > table >  tr:nth-child(4) > td > table"
            )


ImportError: cannot import name 'DB_USER' from 'config.settings' (/Users/main/Project/trustventure/crawling/config/settings.py)

In [2]:
url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
# url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
# page += 1

from utilities import request_helper

req = requests.get(url)
soup = BeautifulSoup(req.content, "lxml", )

In [10]:
!pwd

/Users/main/Project/trust/bs4_ipo/notebooks


In [8]:
!pip install async_retrying

  Preparing metadata (setup.py) ... done
  Created wheel for async_retrying: filename=async_retrying-0.2.2-py3-none-any.whl size=3141 sha256=c645efcef635f5933e110c7ccc3281975f7ffe90b094e673dc043965d8f349c0
  Stored in directory: /Users/main/Library/Caches/pip/wheels/8a/e0/ac/1fc65add82fce6968b0a28a9b7a274385e839467d1a56dd160
Successfully built async_retrying

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [28]:
from datetime import datetime

year = 2023
datetime_str = f'{year}.09.02'
# datetime_str = f'09.02'

datetime_object = datetime.strptime(datetime_str, '%Y.%m.%d') + datetime.timedelta(days=1)

print(type(datetime_object))
print(datetime_object)  # printed in default format
print(datetime.now().strftime('%Y.%m.%d'))
datetime_object == datetime.now().strftime('%m.%d')

<class 'datetime.datetime'>
2023-09-02 00:00:00
2023.02.01


False

In [22]:
None > datetime.now()

TypeError: '>' not supported between instances of 'NoneType' and 'datetime.datetime'